In [3]:
pip install sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install psycopg3-binary

ERROR: Could not find a version that satisfies the requirement psycopg3-binary (from versions: none)
ERROR: No matching distribution found for psycopg3-binary
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install ipython-sql

Note: you may need to restart the kernel to use updated packages.


In [6]:
%load_ext sql
from sqlalchemy import create_engine
%config SqlMagic.style = '_DEPRECATED_DEFAULT'


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [7]:
%sql postgresql://postgres:@localhost:5432/absadatabase

In [ ]:
pip uninstall prettytable


Found existing installation: prettytable 3.12.0
Uninstalling prettytable-3.12.0:
  Would remove:
    /opt/anaconda3/lib/python3.12/site-packages/prettytable-3.12.0.dist-info/*
    /opt/anaconda3/lib/python3.12/site-packages/prettytable/*
Proceed (Y/n)? 

In [ ]:
%%sql
CREATE TABLE ASSETS AS
SELECT 
    transaction_description,
    SUM(CASE WHEN amt < 0 THEN amt ELSE 0 END) AS credit,
    SUM(CASE WHEN amt > 0 THEN amt ELSE 0 END) AS debit,
    SUM(amt) AS balance,
    CASE 
        WHEN SUM(amt) > 0 THEN 'LOSS'
        WHEN SUM(amt) < 0 THEN 'PROFIT'
        ELSE 'NEUTRAL'
    END AS profit_loss
FROM 
    transactiondata
WHERE 
    transaction_description IN (
        'CREDIT TRANSFER',
        'TEL CR TRANSFER',
        'MOBILE PAYMENT CR',
        'DIGITAL PAYMENT CR',
        'ATM PAYMENT FR',
        'CASHSEND ATM',
        'CASHSEND DIGITAL',
        'CASHSEND MOBILE',
        'POS REFUND PUR',
        'POS CARD REFUND',
        'POS O/SEA REFUND',
        'DIGITAL TRANSF CR',
        'INVESTMENT CAPITAL',
        'INVESTMNT INTEREST',
        'INTEREST',
        'INTEREST ADJUST',
        'BAD DEBT W/OFF'
    )
GROUP BY 
    transaction_description;


UsageError: Cell magic `%%sql` not found.
